# **Mount google drive folder**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


# **Import libraries**

In [0]:
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Bidirectional, CuDNNLSTM, LeakyReLU, BatchNormalization, Reshape, Input
from keras.callbacks import ModelCheckpoint
import os
from IPython.display import HTML, display

Using TensorFlow backend.


# **Print the files in the dataset**

In [0]:
print(os.listdir('/content/gdrive/My Drive/MIDI'))

['5_rock_180_beat_4-4.mid', '6_blues-shuffle_134_beat_4-4.mid', '6_funk_91_beat_4-4.mid', '6_funk_80_beat_4-4.mid', '6_dance-disco_120_beat_4-4.mid', '6_jazz-funk_116_fill_4-4.mid', '6_jazz-swing_215_beat_4-4.mid', '6_latin-brazilian-maracatu_96_beat_4-4.mid', '6_neworleans-secondline_99_beat_4-4.mid', '6_rock_60_fill_4-4.mid', '6_rock_86_fill_4-4.mid', '1_jazz-swing_124_beat_4-4.mid', '1_latin-venezuelan-sangueo_101_beat_4-4.mid', '1_funk_92_beat_4-4.mid', '1_funk_95_beat_4-4.mid', '1_funk_80_beat_4-4.mid', '1_latin-chacarera_157_beat_3-4.mid', '6_rock_88_beat_4-4.mid', '1_latin-brazilian-bossa_127_beat_4-4.mid', '1_rock_60_beat_4-4.mid', '1_rock_70_beat_6-8.mid', '1_rock_100_beat_4-4.mid', '1_rock_90_beat_4-4.mid', '1_rock_86_beat_4-4.mid', '1_rock_87_beat_4-4.mid', '1_rock_95_beat_4-4.mid', '1_rock_130_beat_4-4.mid', '1_rock_105_beat_4-4.mid', '1_rock_120_beat_4-4.mid', '1_rock_110_beat_4-4.mid', '1_rock-prog_125_beat_4-4.mid', '1_rock-shuffle_85_beat_4-4.mid', '2_latin-brazilian_10

# **progess()**
Define a function that will display a progress bar so that we can monitor the progress of the parsing

In [0]:
def progress(file, value, max=100):
  return HTML("""
      <p>{file}</p>
      <progress
          value='{value}'
          max='{max}',
          style='width: 100%'
      >
          {value}
      </progress>
  """.format(file=file, value=value, max=max))

out = display(progress(0, 100), display_id=True)

# **Parse the notes from the MIDI files**

In [0]:
def getNotes() :
  DIR = '/content/gdrive/My Drive/MIDI'
  num_files = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
  notes = []
  i = 1
  for file in glob.glob(DIR + '/*.mid'):
      # if 'rock' not in file:
      #     continue

      midi = converter.parse(file)

      # print("Parsing %s" % (i))
      out.update(progress(file, i, num_files))
      i = i + 1

      notes_to_parse = None
      parts = instrument.partitionByInstrument(midi)

      if parts:
          notes_to_parse = parts.parts[0].recurse()
      else:
          notes_to_parse = midi.flat.notes

      for element in notes_to_parse:
          if isinstance(element, note.Note):
              notes.append(str(element.pitch))
          elif isinstance(element, chord.Chord):
              notes.append('.'.join(str(n) for n in element.normalOrder))

      # if i > 15:
      #     break

  out.update(progress('COMPLETE', num_files, num_files))
  with open('notes', 'wb') as filepath:
      # write notes in binary format to filepath
      pickle.dump(notes, filepath)
  return notes

# **Get the notes from the MIDI files**

In [0]:
notes = getNotes()
print(str(len(notes)) + ' Total Notes')
print(str(len(set(notes))) + ' Unique Notes')

83626 Total Notes
208 Unique Notes


# **Set up helper values**


In [0]:
sequence_length = 500
generator_input = 100
total_notes = len(notes)
unique_notes = len(set(notes))
pitchnames = sorted(set(notes))
print(pitchnames)
note_to_int = dict( (note, number) for number, note in enumerate(pitchnames) )
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

['0', '0.1', '0.1.2', '0.1.2.3', '0.1.2.5', '0.1.3', '0.1.4', '0.1.6', '0.2', '0.2.3', '0.2.4', '0.2.4.8', '0.2.5', '0.2.5.8', '0.2.6', '0.2.7', '0.3', '0.3.4', '0.3.5', '0.3.7', '0.4', '0.4.5', '0.4.6', '0.4.7', '0.4.7.8', '0.4.8', '0.5', '0.6', '1.2', '1.2.3', '1.2.3.8', '1.2.5', '1.2.5.8', '1.2.6', '1.3', '1.3.8', '1.4', '1.5', '1.5.8', '1.6', '1.7', '10', '10.0', '10.0.1', '10.0.1.2', '10.0.2', '10.0.3', '10.0.4', '10.1', '10.1.2', '10.11.1.2', '10.11.2', '10.2', '10.2.3', '10.2.4', '10.3', '10.3.4', '11.0', '11.0.2', '11.0.3', '11.0.4', '11.1', '11.1.2', '11.1.2.3', '11.2', '11.3', '11.4', '2', '2.3', '2.3.8', '2.4', '2.4.6', '2.4.8', '2.4.8.10', '2.5', '2.5.8', '2.6', '2.6.8', '2.7', '2.7.8', '2.8', '3.4', '3.4.8', '3.4.8.10', '3.7', '3.7.8', '3.8', '3.8.9', '3.9', '4', '4.10', '4.5', '4.5.8', '4.6', '4.6.8', '4.7', '4.7.8', '4.8', '4.8.10', '4.8.11', '4.8.9', '4.9', '5.10', '5.7', '5.7.0', '5.7.8', '5.8', '5.8.0', '5.8.10', '5.9', '6.11', '6.7', '6.7.0', '6.8', '6.8.0', '6.8.1',

# **Set up training input and targets**

In [0]:
trainx = []
trainy = []

# **Prepare Training Sequences**
Create arrays of *sequence_length* where the first sequence is
 *x* will be *notes[0:sequence_length]* and *notes

*   x[0] will be notes[ 0 : sequence_length ]
*   y[0] will be notes[ sequence_length ]

Remembering that [0 : sequence_length] does not include the *sequence_length*th note. We need to put the next note in the sequence into y


In [0]:
for i in range( 0, total_notes - sequence_length ):
  x_sequence = notes[i : i + sequence_length]
  y_sequence = notes[i + sequence_length]
  trainx.append( [ note_to_int[note] for note in x_sequence ] )
  trainy.append(note_to_int[y_sequence])

n_sequences = len(trainx)
print('Number of sequences: ' + str(n_sequences))
print('Sequence 1: ' + str(trainx[0][0]))
print('Sequence 1 output: ' + str(trainy[0]))

Number of sequences: 83126
Sequence 1: 188
Sequence 1 output: 195


# **Normalize the Sequences**
Normalize the sequences so that each note is between 0 and 1. Convert the output values to categorical one-hot vectors

In [0]:
print(trainx[0][0:10])
trainx = np.reshape(trainx, (n_sequences, sequence_length, 1))
print('trainx shape: ' + str(trainx.shape))

[188, 197, 197, 201, 195, 117, 197, 195, 70, 8]
trainx shape: (83126, 500, 1)


In [0]:
# trainx = (trainx) / float(unique_notes)
trainy = np_utils.to_categorical(trainy)

latent_size = (trainx.shape[1], trainx.shape[2])

print('Index of category: ' + str(np.argmax(trainy[0])))
print(trainx[0][0:10])

Index of category: 195
[[188]
 [197]
 [197]
 [201]
 [195]
 [117]
 [197]
 [195]
 [ 70]
 [  8]]


# **Build the Generator and Discriminator for the GAN**
Discriminator:

In [0]:
discriminator = Sequential()
discriminator.add(CuDNNLSTM(512, input_shape=latent_size, return_sequences=True))
discriminator.add(Bidirectional(CuDNNLSTM(512)))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(128))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(64))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(1, activation='sigmoid'))

discriminator.trainable = False
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

discriminator.summary()






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 500, 512)          1054720   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1024)              4202496   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_____________________

Generator:

In [0]:
generator = Sequential()
generator.add(Dense(256, input_dim=generator_input))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(1024))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(np.prod((trainx.shape[1], trainx.shape[2])), activation='tanh'))
generator.add(Reshape(latent_size))
generator.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_7 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_8 (Dense)              (None, 1024)            

In [0]:
# adversary = Sequential()
# adversary.add(generator)
# adversary.add(discriminator)

# adversary.compile(loss='binary_crossentropy', optimizer='adam')

# discriminator.trainable = False

gan_input = Input(shape=(generator_input,))
fake_seq = generator(gan_input)

gan_output = discriminator(fake_seq)

gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')

gan.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 500, 1)            1202420   
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 5954561   
Total params: 7,156,981
Trainable params: 1,198,836
Non-trainable params: 5,958,145
_________________________________________________________________


# **Feed noise through generator**
generate a random batch from the training data

In [0]:
batch_size = 124
x_batch = trainx[ np.random.randint(0, trainx.shape[0], size=batch_size), : ]
print(x_batch[3])

[[ 45]
 [203]
 [198]
 [ 42]
 [ 90]
 [203]
 [ 42]
 [ 52]
 [198]
 [203]
 [195]
 [188]
 [ 52]
 [198]
 [203]
 [195]
 [188]
 [198]
 [  8]
 [198]
 [195]
 [197]
 [205]
 [ 76]
 [203]
 [  8]
 [205]
 [ 90]
 [189]
 [  8]
 [205]
 [ 90]
 [203]
 [  8]
 [198]
 [205]
 [198]
 [193]
 [  8]
 [205]
 [ 52]
 [188]
 [  8]
 [205]
 [ 90]
 [203]
 [  8]
 [205]
 [ 52]
 [203]
 [  8]
 [ 98]
 [203]
 [  8]
 [205]
 [ 90]
 [189]
 [  8]
 [205]
 [ 90]
 [203]
 [  8]
 [205]
 [ 90]
 [203]
 [  8]
 [198]
 [205]
 [198]
 [201]
 [  8]
 [205]
 [ 93]
 [203]
 [195]
 [197]
 [143]
 [188]
 [203]
 [  8]
 [205]
 [198]
 [198]
 [198]
 [205]
 [198]
 [201]
 [205]
 [198]
 [201]
 [117]
 [ 70]
 [198]
 [  8]
 [205]
 [ 52]
 [203]
 [  8]
 [ 70]
 [189]
 [  8]
 [195]
 [ 70]
 [189]
 [197]
 [195]
 [ 70]
 [189]
 [  8]
 [ 70]
 [189]
 [  8]
 [ 67]
 [189]
 [  8]
 [195]
 [ 70]
 [197]
 [  8]
 [ 70]
 [189]
 [  8]
 [ 70]
 [189]
 [  8]
 [ 67]
 [ 28]
 [  8]
 [195]
 [ 67]
 [189]
 [  8]
 [ 70]
 [197]
 [195]
 [195]
 [ 67]
 [198]
 [ 80]
 [198]
 [205]
 [198]
 [205]

generate fake data from a sequence of noise

In [0]:
prog = display(progress(0, 100), display_id=True)
epochs = 400
steps_per_batch = 32
g_loss = -1

for epoch in range(0, epochs):
  for batch in range(steps_per_batch):
    noise = np.random.uniform( 0, 1, size=[batch_size, 100] )
    fake = generator.predict(noise)
    x_batch = trainx[ np.random.randint(0, trainx.shape[0], size=batch_size), : ]
    x = np.concatenate((x_batch, fake))
    y = np.zeros([2*batch_size, 1])
    y[:batch_size, :] = 0.9
    y[batch_size:, :] = 0
    d_loss = discriminator.train_on_batch(x, y)
    y_gen = np.ones(batch_size)
    g_loss = gan.train_on_batch(noise, y_gen)
    prog.update(progress('Epoch ' + str(epoch+1) + ' (batch ' + str(batch) + ') D loss: ' + str(d_loss), epoch, epochs - 1))
  

In [0]:
noise = np.random.uniform( 0, 1, size=[batch_size, 100] )
# print((noise*float(unique_notes)).astype(int)[0])
# print(noise.shape)
fake = generator.predict(noise)
fake = (fake)*float(unique_notes)
fake = np.absolute(fake.astype(int))
print(fake[0])

[[207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [185]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [207]
 [206]
 [207]
 [206]
 [206]
 [206]
 [206]
 [205]
 [196]
 [206]
 [204]
 [205]
 [198]
 [202]
 [205]
 [205]
 [206]
 [205]
 [206]
 [179]
 [181]
 [203]
 [171]
 [204]
 [196]
 [203]
 [187]
 [182]
 [199]
 [185]
 [165]
 [202]
 [198]
 [186]
 [187]
 [163]
 [176]
 [150]
 [142]
 [101]
 [ 15]
 [170]
 [ 98]
 [ 34]
 [ 48]
 [ 64]
 [ 73]
 [ 90]
 [ 23]
 [ 62]
 [ 38]
 [  2]
 [105]
 [ 94]
 [188]
 [123]
 [ 31]
 [133]
 [160]
 [135]
 [ 55]
 [ 35]
 [  0]
 [ 50]
 [ 61]
 [ 77]
 [ 61]
 [193]
 [152]
 [ 60]
 [163]
 [122]
 [ 75]
 [ 41]
 [  5]
 [172]
 [ 91]
 [110]
 [ 26]
 [ 70]
 [121]
 [172]
 [117]
 [141]
 [  5]
 [ 77]
 [  4]
 [142]
 [ 24]

# Generate MIDI file
Use the notes from the generated fake sequence to create a MIDI file to hear the results

In [0]:
offset = 0
output_notes = []
output = []
# for i in range(len(x_batch[0])):
#   output.append(x_batch[0][i][0])
for i in range(len(fake)):
  output.append(fake[0][i][0])

print(output)
# create note and chord objects based on the values generated by the model
for pattern in output:
    # pattern is a chord
    if pattern > 357:
      pattern = 357
    n = int_to_note[pattern]
    if ('.' in n) or n.isdigit():
        notes_in_chord = n.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(n)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='/content/gdrive/My Drive/GAN DRUM OUTPUT/test_output2.mid')

NameError: ignored